The objective of this notebook is to preprocess the dataset 

## Libraries

In [1]:
import pandas as pd 
import re
from string import punctuation
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer


[nltk_data] Downloading package punkt to /home/maldu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/maldu/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /home/maldu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df = pd.read_csv("../data/bronze/spam.csv")
pd.set_option('display.max_columns', None)
print(df[0:5])

  Category                                            Message
0      ham  Go until jurong point, crazy.. Available only ...
1      ham                      Ok lar... Joking wif u oni...
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...
3      ham  U dun say so early hor... U c already then say...
4      ham  Nah I don't think he goes to usf, he lives aro...


### Change data types

In [3]:
df['Category'] = df['Category'].map({"ham": 0, "spam": 1}).astype(int)
df['Message'] = df['Message'].astype(str)

In [4]:
df_duplicated = df['Message'].duplicated().sum()

In [5]:
df.drop_duplicates(inplace=True)

In [6]:
def clean_text(text):
    special_replacements = {
        r"£": "pound",
        r"\$": "dollar",
        r"\€": "euro",
        r"%": "percentage", 
        r"ì": "i",
        r"ü": "you",
        }
    
    emoticon_pattern = re.compile(r"""
    [:;=Xx]           
    [-~]?             
    [\)\]\(\[dDpP/]   
    """, re.VERBOSE)
    
    for pattern, replacement in special_replacements.items():
        text = re.sub(pattern, replacement, text)
    text = re.sub(emoticon_pattern, 'emoji', text)
    text = text.lower()
    text = re.sub('<[^<>]+>', ' ', text)
    text = re.sub(r'http\S+|www.\S+', '', text)
    text = re.sub('[0-9]+', 'number', text)
    text = re.sub('[^\s]+@[^\s]+', 'emailaddr', text)
    text = text.translate(str.maketrans('', '', punctuation))
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [7]:
df['Message']=df['Message'].apply(clean_text)
df['Message'] = df['Message'].apply(word_tokenize)
df['Message'] = df['Message'].apply(lambda x: [word for word in x if word not in stop_words])

lemmatizer=WordNetLemmatizer()
def lem_tokens(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

df['Message'] = df['Message'].apply(lem_tokens)
df = df[df['Message'].str.strip().astype(bool)]

In [8]:
df.to_csv("../data/silver/df_preprocessed.csv", index= False)
df

,Category,Message
0,0,"[go, jurong, point, crazy, available, bugis, n..."
1,0,"[ok, lar, joking, wif, u, oni]"
2,1,"[free, entry, number, wkly, comp, win, fa, cup..."
3,0,"[u, dun, say, early, hor, u, c, already, say]"
4,0,"[nah, dont, think, go, usf, life, around, though]"
...,...,...
5567,1,"[numbernd, time, tried, number, contact, u, u,..."
5568,0,"[b, going, esplanade, fr, home]"
5569,0,"[pity, mood, soany, suggestion]"
5570,0,"[guy, bitching, acted, like, id, interested, b..."
